In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
burn_filt = Filter.create_filter(address=None, event_types=[Pair.events.Burn])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 1000

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=burn_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])

    if(len(arguments) >= 1):
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['amount0'] = math.nan

    if(len(arguments) >= 2):
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(arguments[1])
    else:
        event['details']['amount1'] = math.nan    
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,310,663 tx:0xc7bcada0c59ca8ec8e90c7c2cee984c2ab3c79fd641ad81c7f702342a12b15c5
Transfer at block:60,310,695 tx:0x828244be72e6766f58d18cf5e84118ff402fd1982fecfcb5e98221ffa9cbc2cb
Transfer at block:60,310,791 tx:0xb08158755dad56ebf37e9b8fd1b0fd83e24399278dd7e6a23de721cb24f35b3a
Transfer at block:60,311,169 tx:0xe4d543f927969a7fe1c626f6e6494bdaac33d9149c21fbdb3156b155784a852f
Transfer at block:60,311,277 tx:0x34d4830e4e2ed9e41e4054d9c8a4e0d0fc8e443fb4b599bedaa5a9997acb17ab
Transfer at block:60,311,534 tx:0x7e80399242f50717f3e8d03c8a1e6b7b6a7aeb11c39349928846035612221b67
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,burn,sushi,0x034293f21f1cce5908bc605ce5850df2b1059ac0,0xc7bcada0c59ca8ec8e90c7c2cee984c2ab3c79fd641a...,60310663,1723057497,{'web3_type': <class 'web3._utils.datatypes.Bu...
1,polygon,uniswapv2pair,burn,sushi,0x6cf8654e85ab489ca7e70189046d507eba233613,0x828244be72e6766f58d18cf5e84118ff402fd1982fec...,60310695,1723057565,{'web3_type': <class 'web3._utils.datatypes.Bu...
2,polygon,uniswapv2pair,burn,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0xb08158755dad56ebf37e9b8fd1b0fd83e24399278dd7...,60310791,1723057769,{'web3_type': <class 'web3._utils.datatypes.Bu...
3,polygon,uniswapv2pair,burn,sushi,0x0e83904f1b79a34a62ebc3bd389a8b35d7c82f2e,0xe4d543f927969a7fe1c626f6e6494bdaac33d9149c21...,60311169,1723058638,{'web3_type': <class 'web3._utils.datatypes.Bu...
4,polygon,uniswapv2pair,burn,sushi,0x5bb9c55ef9b06a1f79d81d91771a7625c0fb72de,0x34d4830e4e2ed9e41e4054d9c8a4e0d0fc8e443fb4b5...,60311277,1723058866,{'web3_type': <class 'web3._utils.datatypes.Bu...
5,polygon,uniswapv2pair,burn,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0x7e80399242f50717f3e8d03c8a1e6b7b6a7aeb11c393...,60311534,1723059446,{'web3_type': <class 'web3._utils.datatypes.Bu...
